# Intro

This notebook is about using one of the famous CNN architecture to solve an image classification task on Cassava Leaf Disease dataset.
It contains a simple code to try training InceptionNet model from torchvision.models and finetune it on this dataset in order to get better result.
Finetuning is concept of transfer learning that uses a pretrained model on huge dataset and try to adapt its weights on the dataset of a specified task. You can either finetune only the last classifier layer, or last N layers or retrain the whole model on the specified dataset.

In this notebook I used a variable train_CNN (False/True) to tell the model if we want to train all CNN model or only the last classifier layer
There are also some parameters you can tweak to get other results (better or worse) but you need to try!!

# Libraries

In [ ]:
import torch
# Creating Dataset
from torch.utils.data import Dataset
import pandas as pd
import os
from PIL import Image

# For creating CNN model and training
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from tqdm import tqdm

# Creating our Dataset

In [ ]:
class LeafDiseaseDataset(Dataset):
    def __init__(self, root_dir, annotation_file, transform=None):
        self.root_dir = root_dir
        self.annotations = pd.read_csv(annotation_file)
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_id = self.annotations.iloc[index, 0]
        img = Image.open(os.path.join(self.root_dir, img_id)).convert("RGB")
        y_label = torch.tensor(float(self.annotations.iloc[index, 1]))

        if self.transform is not None:
            img = self.transform(img)

        return (img, y_label)

# Precise data augmentation (Images transformation)

In [ ]:
transform = transforms.Compose(
        [
            transforms.Resize((356, 356)),
            transforms.RandomCrop((299, 299)),
            transforms.RandomRotation(50),
            transforms.RandomVerticalFlip(0.4), 
            transforms.RandomHorizontalFlip(0.4), 
            transforms.ColorJitter(brightness=0.1, contrast=0.2, saturation=0, hue=0),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ]
    )

# Define the CNN arcitecture (based on InceptionNet)

In [ ]:
class CNN(nn.Module):
    def __init__(self, train_CNN=False, num_classes=5):
        super(CNN, self).__init__()
        self.train_CNN = train_CNN
        self.inception = models.inception_v3(pretrained=True, aux_logits=False)
        self.inception.fc = nn.Linear(self.inception.fc.in_features, num_classes)
        #self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        #self.softmax = nn.Softmax()

    def forward(self, images):
        features = self.inception(images)
        return self.dropout(features)

# Initializations

In [ ]:
num_epochs = 14
learning_rate = 0.0001
train_CNN = True
batch_size = 32
shuffle = True
pin_memory = True
num_workers = 1

device = ("cuda" if torch.cuda.is_available() else "cpu")

# Starting ...

In [ ]:
images_path = "../input/cassava-leaf-disease-classification/train_images"
training_csv_file = "../input/cassava-leaf-disease-classification/train.csv"

In [ ]:
dataset = LeafDiseaseDataset(images_path,training_csv_file,transform=transform)

In [ ]:
len(dataset)

In [ ]:
df = pd.read_csv("../input/cassava-leaf-disease-classification/train.csv")
print(df.head())
print(df.label.unique())

In [ ]:
train_set, validation_set = torch.utils.data.random_split(dataset,[17000,4397])
train_loader = DataLoader(dataset=train_set, shuffle=shuffle, batch_size=batch_size,num_workers=num_workers,pin_memory=pin_memory)
validation_loader = DataLoader(dataset=validation_set, shuffle=shuffle, batch_size=batch_size,num_workers=num_workers, pin_memory=pin_memory)

In [ ]:
#We only use this in the training of the model 
# model = CNN().to(device)

# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# for name, param in model.inception.named_parameters():
#     if "fc.weight" in name or "fc.bias" in name:
#         param.requires_grad = True
#     else:
#         param.requires_grad = train_CNN

In [ ]:
def check_accuracy(loader, model):
    if loader == train_loader:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on validation data")

    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            predictions = torch.tensor(scores).to(device)
            _, predictions = torch.max(predictions, dim=1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
        model.train()
    return f"{float(num_correct)/float(num_samples)*100:.2f}"   

In [ ]:
def train():
    model.train()
    for epoch in range(num_epochs):
        losses = []
        for imgs, labels in train_loader:
            imgs = imgs.to(device)
            labels = labels.to(device, dtype=torch.long)
            outputs = model(imgs)
            predictions = torch.tensor(outputs).to(device)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            losses.append(loss.item())
        if epoch % 2 == 0:
            val_acc = check_accuracy(validation_loader, model)
            print(val_acc)
        print(f"Epoch [{epoch}/{num_epochs}] ===> ", sum(losses)/len(losses))

# We will not train it here, I will use my pretrained model insteasd
# if __name__ == "__main__":
#     train()

# PATH = "CassavleafDiseaseClassificationPretrainedInceptionV3.pkl"
# #Store the model after training it in a path to be used in the testing phase
# torch.save(model, PATH)

# Apply the trained CNN model (Inception) on the test data

In [ ]:
PATH = "../input/casavadiseaseincetionv3model/CassavleafDiseaseClassificationPretrainedInceptionV3.pkl"
model = torch.load(PATH)
model.eval()

In [ ]:
test_images_path = "../input/cassava-leaf-disease-classification/test_images"
submission_file = "../input/cassava-leaf-disease-classification/sample_submission.csv"

In [ ]:
class TestLeafDiseaseDataset(Dataset):
    def __init__(self, root_dir, image_names_file, transform=None):
        self.root_dir = root_dir
        self.image_names = pd.read_csv(image_names_file)
        self.transform = transform

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, index):
        img_id = self.image_names.iloc[index, 0]
        img = Image.open(os.path.join(self.root_dir, img_id)).convert("RGB")

        if self.transform is not None:
            img = self.transform(img)

        return img

In [ ]:
test_set = TestLeafDiseaseDataset(test_images_path, submission_file, transform=transform)

In [ ]:
len(test_set)

In [ ]:
test_loader = DataLoader(dataset=test_set, batch_size=batch_size,num_workers=num_workers,pin_memory=pin_memory)

# Create the submission file

In [ ]:
def test(loader, model):
    model.eval()
    predicted = []
    with torch.no_grad():
        for x in loader:
            x = x.to(device=device)

            scores = model(x)
            predictions = torch.tensor(scores).to(device)
            _, predictions = torch.max(predictions, dim=1)
            predicted.extend(predictions.cpu().numpy())
    return predicted

In [ ]:
pred = test(test_loader, model)

In [ ]:
submission = pd.read_csv("../input/cassava-leaf-disease-classification/sample_submission.csv")
submission.label = pred

In [ ]:
submission.head()

In [ ]:
submission.to_csv('submission.csv', index=False)